In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
import sys

In [16]:
class EmbModel(tf.keras.Model):
    def __init__(self, useridlength, category_length):
        super(EmbModel, self).__init__()
        self.d_steps = 1
        self.useridlength = useridlength
        self.category_length = category_length
        self.model = self.init_model()
        print(self.useridlength)
        
    def call(self, inputs):
        return
    
    def get_config(self):
        return {"hidden_units": self.hidden_units}
    
    def init_model(self):
        poi_latitude_input = keras.layers.Input(shape=(1,), name='poi_latitude')
        poi_longitude_input = keras.layers.Input(shape=(1,), name='poi_longitude')
        poi_concat_input = tf.keras.layers.Concatenate(axis=-1)([poi_latitude_input, poi_longitude_input])
        #input_length:  #This is the length of input sequences, as you would define for any input layer of a Keras model. 
                        #For example, if all of your input documents are comprised of 1000 words, this would be 1000
        #input_dim: 
                        #This is the size of the vocabulary in the text data. 
                        #For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
        poi_dense = keras.layers.Dense(8)(poi_concat_input)
        poi_reshape = keras.layers.Reshape((1, 8))(poi_dense)
        
        category_input = keras.layers.Input(shape=(1), name='category_input')
        category_emb = keras.layers.Embedding(self.category_length, 8)(category_input)    
        category_concat = tf.keras.layers.Concatenate(axis=-1)([category_emb, poi_reshape])
    
        user_input = keras.layers.Input(shape=(1,), name='user_id')
        user_emb = keras.layers.Embedding(self.useridlength, 16)(user_input)
        #user_reshape = layers.Reshape((1, 256))(user_emb)
                                    
        dot = keras.layers.Dot(axes=(2))([category_concat, user_emb])
            
        model = Model([category_input, poi_latitude_input, poi_longitude_input, user_input], dot)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        if len(data) == 3:
            real_data, labels, sample_weight = data
        else:
            sample_weight = None
            real_data, labels = data
        cat_data = real_data[0]
        lat_data = real_data[1]
        long_data = real_data[2]
        user_data = real_data[3]

        for i in range(self.d_steps):
            with tf.GradientTape() as tape:
                
                #print(latlong_data[0])
                #print(latlong_data[1])
                #print(user_data)
                
                dotproduct = self.model([cat_data, lat_data, long_data, user_data])
                #print(dotproduct)
                # Loss function = ||S-GroundTruth|| 
                loss = tf.math.abs(tf.subtract(tf.cast(dotproduct, tf.float64), labels))
                #print(loss)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))
        return {'loss': loss}
    
    def predict_step(self, data):
        sample_weight = None
        cat_data = real_data[0]
        lat_data = real_data[1]
        long_data = real_data[2]
        user_data = real_data[3]
        return self.model([cat_data, lat_data, long_data, user_data])

In [20]:
model = EmbModel(1, 1)
model.build((None, 4))
model.model.save(r'C:\Users\lasse\Desktop\RecommenderDL\Kode', save_format="tf") 

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_latitude (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_longitude (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_22 (Concatenate)    (None, 2)            0           poi_latitude[0][0]               
                                                                 poi_longitude[0][0]              
__________________________________________________________________________________________________
category_input (InputLayer)     [(None, 1)]          0                                

In [27]:
assert imported([tf.constant(3.), tf.constant(3.), tf.constant(3.), tf.constant(3.)]).numpy() == 3

TypeError: '_UserObject' object is not callable

In [8]:
model = tf.saved_model.load_model(r'C:\Users\lasse\Desktop\RecommenderDL\models\model64')

AttributeError: module 'tensorflow._api.v2.saved_model' has no attribute 'load_model'

In [21]:
imported.predict()

AttributeError: '_UserObject' object has no attribute 'predict'

In [4]:
x_test_df = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\y_train_df_64.csv', sep=',', names=['User','Latitude','Longitude', '0'], encoding='latin-1').dropna(axis=1)
y_test_df = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\y_test_df_64.csv', sep=',', names=['0'], encoding='latin-1').dropna(axis=1)

In [5]:

for index, row in x_test_df.iterrows():

    
    long = tf.convert_to_tensor([x_test_df.loc[index, 'User']],dtype=None, dtype_hint=None, name=None)
    lat = tf.convert_to_tensor([x_test_df.loc[index, 'Latitude']],dtype=None, dtype_hint=None, name=None)
    user = tf.convert_to_tensor([x_test_df.loc[index, 'Longitude']],dtype=None, dtype_hint=None, name=None)
    cat = tf.convert_to_tensor([x_test_df.loc[index, '0']],dtype=None, dtype_hint=None, name=None)
    visited = ground_truth = y_test_df.loc[index, '0']
    score = model.predict([cat, lat, long, user])
    print("Score:", score[0][0])
    if visited == 1:
        print("The user has visited the PoI")
    else:
        print("The user has not visited the PoI \n")

NameError: name 'model' is not defined